# ETL Pipeline Preparation

#### The second project due as part of Udacity's Data Science Nanodegree involves building an app that can use a NLP model to classify messages relating to natural disasters.

#### There are three stages to this project:

#### 1) Extracting and cleaning data
#### 2) Using a machine learning pipeline to create a model for classifying data
#### 3) Deploying the model as a web app using Flask.

#### This notebook relates to step 1 and goes through the data preparation process.

### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import re
from sqlalchemy import create_engine

In [2]:
messages = pd.read_csv("D:\\lori_python\\Udacity_Projects-main\\Udacity_Projects-main\\NLP project\\disaster_messages.csv")

In [3]:
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
categories = pd.read_csv("D:\\lori_python\\Udacity_Projects-main\\Udacity_Projects-main\\NLP project\\disaster_categories.csv")
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [5]:
df = messages.merge(categories, on=('id'))
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns

#### Split the values in the `categories` column on the `;` character so that each value becomes a separate column.

In [6]:
categories = df['categories'].str.split(";", expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


#### Use the first row of categories dataframe to create column names for the categories data.

In [7]:
row = categories.iloc[[0]]

In [8]:
row

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


#### Use this row to extract a list of new column names for categories. One way is to apply a lambda function that takes everything up to the second to last character of each string with slicing

In [9]:
category_colnames = row.apply(lambda x: x.str.slice(stop=-2))
print(category_colnames)

        0        1      2            3             4                 5   \
0  related  request  offer  aid_related  medical_help  medical_products   

                  6         7         8            9   ...           26  \
0  search_and_rescue  security  military  child_alone  ...  aid_centers   

                     27               28      29     30    31          32  \
0  other_infrastructure  weather_related  floods  storm  fire  earthquake   

     33             34             35  
0  cold  other_weather  direct_report  

[1 rows x 36 columns]


In [10]:
category_colnames

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [11]:
print(category_colnames.shape)

(1, 36)


In [12]:
print(categories.shape)

(26386, 36)


In [13]:
category_colnames = category_colnames.values.tolist()

In [14]:
category_colnames

[['related',
  'request',
  'offer',
  'aid_related',
  'medical_help',
  'medical_products',
  'search_and_rescue',
  'security',
  'military',
  'child_alone',
  'water',
  'food',
  'shelter',
  'clothing',
  'money',
  'missing_people',
  'refugees',
  'death',
  'other_aid',
  'infrastructure_related',
  'transport',
  'buildings',
  'electricity',
  'tools',
  'hospitals',
  'shops',
  'aid_centers',
  'other_infrastructure',
  'weather_related',
  'floods',
  'storm',
  'fire',
  'earthquake',
  'cold',
  'other_weather',
  'direct_report']]

#### rename the columns of `categories`

In [15]:
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.

#### Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. The method I've used replaces anything that isn't numerical with blank space


In [16]:
for column in categories:
 
    categories[column] = categories[column].str.replace("\D+", '')
                                                       

C:\Users\Laurence\AppData\Local\Temp\ipykernel_18844\1557981413.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  categories[column] = categories[column].str.replace("\D+", '')


#### When working through the project initially, column names showed as tuples after joining 'categories' and 'df'. Let's join the words within each name together.

In [17]:
categories.columns = categories.columns.map('_'.join)

#### Convert column values from string to numeric

In [18]:
categories = categories.astype(int)

In [19]:
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26386 entries, 0 to 26385
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   related                 26386 non-null  int32
 1   request                 26386 non-null  int32
 2   offer                   26386 non-null  int32
 3   aid_related             26386 non-null  int32
 4   medical_help            26386 non-null  int32
 5   medical_products        26386 non-null  int32
 6   search_and_rescue       26386 non-null  int32
 7   security                26386 non-null  int32
 8   military                26386 non-null  int32
 9   child_alone             26386 non-null  int32
 10  water                   26386 non-null  int32
 11  food                    26386 non-null  int32
 12  shelter                 26386 non-null  int32
 13  clothing                26386 non-null  int32
 14  money                   26386 non-null  int32
 15  missing_people     

### 5. Replace `categories` column in `df` with new category columns.

#### drop the original categories column from `df`

In [21]:
df = df.drop('categories', axis = 1)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


#### concatenate the original dataframe with the new `categories` dataframe

In [22]:
df = df.join(categories) 
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates

#### Check how many duplicates are in this dataset.

In [23]:
len(df) - df['id'].nunique() 

206

#### Drop the duplicates.

In [24]:
df = df.drop_duplicates(subset=['id'])

#### Confirm duplicates were removed.

In [25]:
len(df) - df['id'].nunique() 

0

In [26]:
len(df)

26180

In [29]:
df['id'].nunique() 

26180

### 7. Save the clean dataset into an sqlite database.
#### This is done using the pandas [`to_sql`] method. In the first line an engine in created to connect with the desired file location

In [30]:
engine = create_engine('sqlite:///NLPproject.db')
df.to_sql('NLPtraining', engine, index=False)

26180

### 8. Use this notebook to complete `process_data.py`
#### The cells below present functions that can conveniently execute the steps above for similar datasets.
#### The final cell presents code that executes these functions on our disaster response data in the 'process_data.py' file

In [ ]:
###
This function loads in two datasets - one containing messages from diaster zones and another with category dummies that show what the message is about.
When working in Flask the data is stored in a directory, which we can access by stating the filepath where each dataset is saved.

Inputs: messages_filepath (str), categories_filepath (str)

Outputs: df (dataframe), categories (dataframe) - df is a dataframe combining botht the messages and categories datasets. 
We also return the original categories dataset so that we can use it to clean the df dataframe.
###

def load_data(messages_filepath, categories_filepath):
    
 messages = pd.read_csv(messages_filepath)

 categories = pd.read_csv(categories_filepath)

 df = messages.merge(categories, on=('id')) 

 return df, categories

In [ ]:
###
This function is used to create a matrix of columns with names showing what category the column relates to. 
It also converts the categories data in dummy variables that can be passed to a machine learning pipeline later on.
This matrix is then used to replace the 'categories' column in the df dataframe

Inputs - df (dataframe)
Outputs - df (dataframe) - this is the df dataframe with the categories matrix as described above
###

def clean_data(df):
 categories = df['categories'].str.split(";", expand=True)
 row = categories.iloc[[0]]
 category_colnames = row.apply(lambda x: x.str.slice(stop=-2))
 category_colnames = category_colnames.values.tolist()
 categories.columns = category_colnames

 for column in categories:
     categories[column] = categories[column].str.replace("\D+", '')
    
 categories.columns = categories.columns.map('_'.join)
 categories = categories.astype(int)
 df = df.drop('categories', axis = 1)
 df = df.join(categories) 
 df = df.drop_duplicates(subset=['id'])

 return df

In [ ]:
###
This function saves the cleaned data as an sql database to a filepath of your choice.
The table containing the data will be called 'NLPtraining'.

Inputs - df (dataframe)
database_filepath(str) - filepath to the database where the data will be saved. This should include the name of the database with a .db extension
###

def save_data(df, database_filepath): 
engine = create_engine(database_filepath)
df.to_sql('NLPtraining', engine, index=False, if_exists='replace')

In [ ]:
###
This function exectutes all of the above functions
The arguments for the functions are as described above
###


def main():
    if len(sys.argv) == 4:

        messages_filepath, categories_filepath, database_filepath = sys.argv[1:]

        print('Loading data...\n    MESSAGES: {}\n    CATEGORIES: {}'
              .format(messages_filepath, categories_filepath))
        df, categories = load_data("data/disaster_messages.csv", "data/disaster_categories.csv")

        print('Cleaning data...')
        df = clean_data(df)
        
        print('Saving data...\n    DATABASE: {}'.format(database_filepath))
        save_data(df, 'sqlite:///data/DisasterResponse.db')
        
        print('Cleaned data saved to database!')
    
    else:
        print('Please provide the filepaths of the messages and categories '\
              'datasets as the first and second argument respectively, as '\
              'well as the filepath of the database to save the cleaned data '\
              'to as the third argument. \n\nExample: python process_data.py '\
              'disaster_messages.csv disaster_categories.csv '\
              'DisasterResponse.db')


if __name__ == '__main__':
    main()